# Contexte :
# Analyse exploratoire du dataset supply chain pour évaluer le risque et la résilience logistique.

| Variable                          | Description courte                                           | Type    | Exemples                  |
| --------------------------------- | ------------------------------------------------------------ | ------- | ------------------------- |
| `warehouse_inventory_level`       | Niveau de stock en entrepôt pour le produit concerné         | float64 | 985.72, 396.70            |
| `handling_equipment_availability` | Disponibilité des équipements de manutention                 | float64 | 0.48, 0.62                |
| `order_fulfillment_status`        | Niveau d'exécution de la commande                            | float64 | 0.76, 0.19                |
| `weather_condition_severity`      | Sévérité des conditions météo pour l’itinéraire              | float64 | 0.36, 0.23                |
| `shipping_costs`                  | Coût logistique associé à la livraison                       | float64 | 456.50, 640.40            |
| `supplier_reliability_score`      | Fiabilité du fournisseur (0 à 1)                             | float64 | 0.98, 0.46                |
| `lead_time_days`                  | Délai planifié (jours)                                       | float64 | 2.12, 12.60               |
| `historical_demand`               | Demande historique pour ce produit                           | float64 | 100.77, 5313.73           |
| `cargo_condition_status`          | État général du cargo                                        | float64 | 0.77, 0.09                |
| `route_risk_level`                | Niveau de risque de l’itinéraire (0–10)                      | float64 | 1.18, 9.61                |
| `customs_clearance_time`          | Temps moyen de dédouanement                                  | float64 | 0.50, 0.96                |
| `disruption_likelihood_score`     | Probabilité de perturbation (0–1)                            | float64 | 0.50, 0.98                |
| `delay_probability`               | Probabilité de retard (0–1)                                  | float64 | 0.88, 0.54                |
| `delivery_time_deviation`         | Écart réel de livraison (positif = retard, négatif = avance) | float64 | 9.11, 8.17                |
| `product_id`                      | Identifiant unique du produit                                | object  | 'P0353', 'P0857'          |
| `supplier_id`                     | Identifiant unique du fournisseur                            | object  | 'P0353\_S1', 'P0857\_S2'  |
| `supplier_country`                | Pays d’origine du fournisseur                                | object  | Greece, South Korea, Iran |
| `Risk_Score`                      | Indicateur agrégé de risque logistique \[0–1]                | float64 | 0.60, 0.82                |
| `Resilience_Index`                | Indice de résilience globale \[0–1]                          | float64 | 0.38, 0.08                |

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.api.types import CategoricalDtype
import plotly.express as px

# Charger le dataset
file_path = "../data/extracted/Supply_chain_dataset/dynamic_supply_chain_logistics_dataset_with_country.csv"
df = pd.read_csv(file_path)

# Afficher forme, colonnes, premiers exemples
print(f"Shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
df.head()

In [ ]:
# Vérifier types + valeurs manquantes + premières stats pour les variables clés

# 1) Types et infos globales
df.info()

# 2) Valeurs manquantes
print("\nValeurs manquantes :")
print(df.isnull().sum())

# 3) Stats descriptives sur les variables liées au risque/délai/résilience
cols_risk = [
    'route_risk_level', 'disruption_likelihood_score', 'delay_probability',
    'risk_classification', 'delivery_time_deviation',
    'lead_time_days', 'supplier_reliability_score'
]

df[cols_risk].describe(include='all')


In [ ]:
# 1) Vérifier les modalités uniques de risk_classification
print("Valeurs uniques de risk_classification :")
print(df['risk_classification'].unique())

# 2) Vérifier valeurs négatives pour delivery_time_deviation
print("\nValeurs négatives de delivery_time_deviation :")
print(df[df['delivery_time_deviation'] < 0]['delivery_time_deviation'].describe())

# 3) Vérifier les doublons
print("\nNombre de doublons sur (product_id, supplier_id) :")
print(df.duplicated(subset=['product_id', 'supplier_id']).sum())


In [ ]:
# 1) Vérifier si les doublons sont strictement identiques sur TOUTES les colonnes
print("Nombre de doublons stricts (toutes colonnes) :")
print(df.duplicated().sum())

# 2) Exemple de doublons non stricts si existants
if df.duplicated().sum() != df.duplicated(subset=['product_id', 'supplier_id']).sum():
    print("\nExemple de doublons non stricts :")
    dup_subset = df[df.duplicated(subset=['product_id', 'supplier_id'], keep=False)]
    display(dup_subset.head(10))


In [ ]:
# 1) Convertir risk_classification en catégorie ordonnée
risk_order = CategoricalDtype(categories=["Low Risk", "Moderate Risk", "High Risk"], ordered=True)
df['risk_classification'] = df['risk_classification'].astype(risk_order)

# 2) Vérifier le typage final
print(df['risk_classification'].dtype)
df['risk_classification'].value_counts()


In [ ]:
# Vérifier les min/max pour les indicateurs de risque, délai, résilience

risk_cols = [
    'route_risk_level',
    'disruption_likelihood_score',
    'delay_probability',
    'delivery_time_deviation',
    'lead_time_days',
    'supplier_reliability_score'
]

print(df[risk_cols].describe())

# Vérifier visuellement la distribution de chaque variable (boxplot)
plt.figure(figsize=(15, 8))
for i, col in enumerate(risk_cols, 1):
    plt.subplot(2, 3, i)
    sns.boxplot(y=df[col])
    plt.title(col)
plt.tight_layout()
plt.show()


In [ ]:
# Vérifier la queue basse de disruption_likelihood_score
low_disruption = df[df['disruption_likelihood_score'] < 0.1]
print(f"Lignes avec disruption_likelihood_score < 0.1 : {len(low_disruption)}")
low_disruption.describe()

In [ ]:
# Forcer les bornes techniques pour éviter des fuites de valeurs aberrantes dans le pipeline

# Définir les bornes théoriques
bounds = {
    'route_risk_level': (0, 10),
    'disruption_likelihood_score': (0, 1),
    'delay_probability': (0, 1),
    'supplier_reliability_score': (0, 1),
    'lead_time_days': (0, None),  # pas de max fixé
    'delivery_time_deviation': (None, None)  # pas de bornage ici, signe important
}

# Appliquer les bornes
for col, (min_val, max_val) in bounds.items():
    if min_val is not None:
        df[col] = df[col].clip(lower=min_val)
    if max_val is not None:
        df[col] = df[col].clip(upper=max_val)

print("Bornage technique appliqué sur les variables clés.")

In [ ]:
# Normaliser route_risk_level et delivery_time_deviation
df['route_risk_level_norm'] = df['route_risk_level'] / 10
df['delivery_time_deviation_norm'] = df['delivery_time_deviation'] / 10

# Créer Risk_Score (pondérations égales pour commencer)
df['Risk_Score'] = (
    0.25 * df['route_risk_level_norm'] +
    0.25 * df['disruption_likelihood_score'] +
    0.25 * df['delay_probability'] +
    0.25 * df['delivery_time_deviation_norm']
)

# Créer Resilience_Index
df['Resilience_Index'] = df['supplier_reliability_score'] * (1 - df['Risk_Score'])

# Vérifier
print(df[['Risk_Score', 'Resilience_Index']].describe())


In [ ]:
# Histogramme Risk_Score
plt.figure(figsize=(12, 5))
sns.histplot(df['Risk_Score'], bins=50, kde=True, color='red')
plt.title("Distribution du Risk_Score")
plt.xlabel("Risk_Score")
plt.show()

# Histogramme Resilience_Index
plt.figure(figsize=(12, 5))
sns.histplot(df['Resilience_Index'], bins=50, kde=True, color='green')
plt.title("Distribution du Resilience_Index")
plt.xlabel("Resilience_Index")
plt.show()

In [ ]:
# DataFrame final pour export ou dashboard
cols_final = ['product_id', 'supplier_id', 'supplier_country', 'Risk_Score', 'Resilience_Index']
df_final = df[cols_final].copy()

print(df_final.head())
print(df_final.describe())

In [ ]:
# Sélectionner les colonnes essentielles pour le livrable final
cols_final = [
    'product_id', 'supplier_id', 'supplier_country',
    'route_risk_level',
    'disruption_likelihood_score',
    'delay_probability',
    'delivery_time_deviation',
    'lead_time_days',
    'supplier_reliability_score',
    'Risk_Score',
    'Resilience_Index'
]

df_final = df[cols_final].copy()

print(df_final.head())
print(df_final.describe())

In [ ]:
# 1) Prendre la MOYENNE du Risk_Score par pays
df_country_risk = df_final.groupby("supplier_country", as_index=False)["Risk_Score"].mean()

# 2) Vérifier la plage réelle
min_risk = df_country_risk["Risk_Score"].min()
max_risk = df_country_risk["Risk_Score"].max()

print(f"Min Risk_Score (mean per country): {min_risk:.3f}")
print(f"Max Risk_Score (mean per country): {max_risk:.3f}")

# 3) Carte des pays les plus sûrs (faible moyenne de risque)
fig_low_risk = px.choropleth(
    df_country_risk,
    locations="supplier_country",
    locationmode="country names",
    color="Risk_Score",
    color_continuous_scale="Greens_r",
    range_color=(min_risk, max_risk),
    title="Countries with Lowest Average Risk Score"
)

fig_low_risk.update_geos(showcountries=True, showcoastlines=True, showland=True, fitbounds="locations")
fig_low_risk.update_layout(coloraxis_colorbar=dict(title="Risk Score (Mean)"))
fig_low_risk.show()

# 4) Carte des pays les plus risqués (moyenne de risque élevée)
fig_high_risk = px.choropleth(
    df_country_risk,
    locations="supplier_country",
    locationmode="country names",
    color="Risk_Score",
    color_continuous_scale="Reds",
    range_color=(min_risk, max_risk),
    title="Countries with Highest Average Risk Score"
)

fig_high_risk.update_geos(showcountries=True, showcoastlines=True, showland=True, fitbounds="locations")
fig_high_risk.update_layout(coloraxis_colorbar=dict(title="Risk Score (Mean)"))
fig_high_risk.show()

In [ ]:
# Enregistrer la version nettoyée 
df_final.to_csv("../data/cleaned/supply_chain_cleaned.csv", index=False)
print("Fichier enregistré : /data/cleaned/supply_chain_cleaned.csv")